# The tuned lens 🔎
A tuned lens allows us to peak at the iterative computations that a transformer is using the compute the next token.

A lens into a transformer with n layers allows you to replace the last $m$ layers of the model with an [affine transformation](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) (we call these affine translators).

This essentially skips over these last few layers and lets you see the best prediction that can be made from the model's representations, i.e. the residual stream, at layer $n - m$. Since the representations may be rotated, shifted, or stretched from layer to layer it's useful to train the len's affine translators specifically on each layer. This training is what differentiates this method from simpler approaches that decode the residual stream of the network directly using the unembedding layer i.e. the logit lens. We explain this process along with more applications of the method in [the paper](ttps://arxiv.org/abs/2303.08112).

You can find the complete set of pretrained lenses on [the hugging face space](https://huggingface.co/spaces/AlignmentResearch/tuned-lens/tree/main/lens).

## Usage
Since the tuned lens produces a distribution of predictions to visualize it's output we need to we need to provide a summary statistic to plot.  The default is simply [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)), but you can also choose the [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) with the target token, or the [KL divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between the model's predictions and the tuned lens' predictions. You can also hover over a token to see more of the distribution i.e. the top 10 most probable tokens and their probabilities.

## Examples
Some interesting examples you can try.

### Copy paste:
```
Copy: A!2j!#u&NGApS&MkkHe8Gm!#
Paste: A!2j!#u&NGApS&MkkHe8Gm!#
```

### Trivial in-context learning
```
inc 1 2
inc 4 5
inc 13 
```

#### Addition
```
add 1 1 2
add 3 4 7
add 13 2 
```

In [1]:
# !pip install tuned-lens
# !pip install nbformat --upgrade

In [2]:
# import torch
# from tuned_lens.nn.lenses import TunedLens, LogitLens
# from transformers import AutoModelForCausalLM, AutoTokenizer


# device = torch.device('cpu')
# model_id_pythia = 'gpt2-large'
# cache_dir_pythia = "./models/" + model_id_pythia
# # To try a diffrent modle / lens check if the lens is avalible then modify this code
# model = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path=model_id_pythia,
#     cache_dir=cache_dir_pythia,
#     device_map=device,
#     )
# model = model.to(device)
# tokenizer = AutoTokenizer.from_pretrained(
#     pretrained_model_name_or_path=model_id_pythia,
#     cache_dir=cache_dir_pythia,
#     )
# tuned_lens = TunedLens.from_model_and_pretrained(model)
# tuned_lens = tuned_lens.to(device)
# logit_lens = LogitLens.from_model(model)

In [3]:
# from tuned_lens.plotting import PredictionTrajectory
# import ipywidgets as widgets
# from plotly import graph_objects as go


# def make_plot(lens, text, layer_stride, statistic, token_range):
#     input_ids = tokenizer.encode(text)
#     targets = input_ids[1:] + [tokenizer.eos_token_id]

#     if len(input_ids) == 0:
#         return widgets.Text("Please enter some text.")
    
#     if (token_range[0] == token_range[1]):
#         return widgets.Text("Please provide valid token range.")
#     pred_traj = PredictionTrajectory.from_lens_and_model(
#         lens=lens,
#         model=model,
#         input_ids=input_ids,
#         tokenizer=tokenizer,
#         targets=targets,
#     ).slice_sequence(slice(*token_range))

#     return getattr(pred_traj, statistic)().stride(layer_stride).figure(
#         title=f"{lens.__class__.__name__} ({model.name_or_path}) {statistic}",
#     )

# style = {'description_width': 'initial'}
# statistic_wdg = widgets.Dropdown(
#     options=[
#         ('Entropy', 'entropy'),
#         ('Cross Entropy', 'cross_entropy'),
#         ('Forward KL', 'forward_kl'),
#     ],
#     description='Select Statistic:',
#     style=style,
# )
# text_wdg = widgets.Textarea(
#     description="Input Text",
#     value="it was the best of times, it was the worst of times",
# )
# lens_wdg = widgets.Dropdown(
#     options=[('Tuned Lens', tuned_lens), ('Logit Lens', logit_lens)],
#     description='Select Lens:',
#     style=style,
# )

# layer_stride_wdg = widgets.BoundedIntText(
#     value=2,
#     min=1,
#     max=10,
#     step=1,
#     description='Layer Stride:',
#     disabled=False
# )

# token_range_wdg = widgets.IntRangeSlider(
#     description='Token Range',
#     min=0,
#     max=1,
#     step=1,
#     style=style,
# )


# def update_token_range(*args):
#     token_range_wdg.max = len(tokenizer.encode(text_wdg.value))

# update_token_range()

# token_range_wdg.value = [0, token_range_wdg.max]
# text_wdg.observe(update_token_range, 'value')

# interact = widgets.interact.options(manual_name='Run Lens', manual=True)

# plot = interact(
#     make_plot,
#     text=text_wdg,
#     statistic=statistic_wdg,
#     lens=lens_wdg,
#     layer_stride=layer_stride_wdg,
#     token_range=token_range_wdg,
# )

In [4]:
import torch
from tuned_lens.nn.lenses import TunedLens, LogitLens
from tuned_lens import TunedLens
# from tuned_lens_example import TunedLens, LogitLens
from transformers import AutoModelForCausalLM, AutoTokenizer

from tuned_lens.plotting import PredictionTrajectory
import ipywidgets as widgets
from plotly import graph_objects as go

# temp = torch.load("/Users/s2011847/.cache/huggingface/hub/spaces--AlignmentResearch--tuned-lens/snapshots/1ac7285852a22309f571c2555efc37375d0c4cda/lens/EleutherAI/pythia-410m-deduped/params.pt", map_location=torch.device("cpu"))

device = torch.device('mps')
# model_id_pythia = 'gpt2-large'
# model_id_pythia = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id_pythia = "EleutherAI/pythia-410m-deduped"
if "llama" not in model_id_pythia: cache_dir_pythia = "./models/" + model_id_pythia
# To try a diffrent modle / lens check if the lens is avalible then modify this code
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_id_pythia,
    cache_dir=cache_dir_pythia,
    device_map=device,
    )
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_id_pythia,
    cache_dir=cache_dir_pythia,
    )
tuned_lens = TunedLens.from_model_and_pretrained(model)
tuned_lens = tuned_lens.to(device)
logit_lens = LogitLens.from_model(model)


def make_plot(lens, text, layer_stride, statistic, token_range):
    input_ids = tokenizer.encode(text)
    targets = input_ids[1:] + [tokenizer.eos_token_id]

    if len(input_ids) == 0:
        return widgets.Text("Please enter some text.")
    
    if (token_range[0] == token_range[1]):
        return widgets.Text("Please provide valid token range.")
    pred_traj = PredictionTrajectory.from_lens_and_model(
        lens=lens,
        model=model,
        input_ids=input_ids,
        tokenizer=tokenizer,
        targets=targets,
    ).slice_sequence(slice(*token_range))

    return getattr(pred_traj, statistic)().stride(layer_stride).figure(
        title=f"{lens.__class__.__name__} ({model.name_or_path}) {statistic}",
    )

style = {'description_width': 'initial'}
statistic_wdg = widgets.Dropdown(
    options=[
        ('Entropy', 'entropy'),
        ('Cross Entropy', 'cross_entropy'),
        ('Forward KL', 'forward_kl'),
    ],
    description='Select Statistic:',
    style=style,
)
text_wdg = widgets.Textarea(
    description="Input Text",
    value="it was the best of times, it was the worst of times",
)
lens_wdg = widgets.Dropdown(
    options=[('Tuned Lens', tuned_lens), ('Logit Lens', logit_lens)],
    description='Select Lens:',
    style=style,
)

layer_stride_wdg = widgets.BoundedIntText(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Layer Stride:',
    disabled=False
)

token_range_wdg = widgets.IntRangeSlider(
    description='Token Range',
    min=0,
    max=1,
    step=1,
    style=style,
)


def update_token_range(*args):
    token_range_wdg.max = len(tokenizer.encode(text_wdg.value))

update_token_range()

token_range_wdg.value = [0, token_range_wdg.max]
text_wdg.observe(update_token_range, 'value')

interact = widgets.interact.options(manual_name='Run Lens', manual=True)

plot = interact(
    make_plot,
    text=text_wdg,
    statistic=statistic_wdg,
    lens=lens_wdg,
    layer_stride=layer_stride_wdg,
    token_range=token_range_wdg,
)

/Users/s2011847/repos/tuned-lens/tuned_lens/nn/lenses.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = th.load(ckpt_path, **th_load_kwargs, map_location=device)

interactive(children=(Dropdown(description='Select Lens:', options=(('Tuned Lens', TunedLens(
  (unembed): Une…